6 1
10 20 10 30 20 50
5 1
3 5 2 6 1
5 1
1 4 2 3 5
5 2
1 4 2 3 5
6 1
2 6 4 1 3 5
6 2
2 6 4 1 3 5
6 3
2 6 4 1 3 5
6 4
2 6 4 1 3 5
6 6
1 1 2 3 2 3

- 시간복잡도 문제로 풀지 못했다.
### 최단거리 index의 의미 해석(18892) - 잘못된 해석
- index 리스트를 거꾸로 보면서 LIS 길이에서 1씩 줄여가며 첫번째로 보이는 index가 가리키는 원소를 출력했다.
- 이렇게 거꾸로가는 iteration 중에 감소하는 순서로 index를 선택하기만 한다면 모든 LIS를 출력할 수 있을 것으로 보인다.
- `DP[x][y]` = index리스트를 거꾸로 보면서, x보다 앞에 있으면서 LIS를 구성할 수 있는, 뒤에서 y번째 숫자인 인덱스들

In [ ]:
import bisect
N, K = map(int, input().split())
DATA = [*map(int, input().split())]
l = [0] * N
LIS = [-float("inf")]
MAX = 0 

for i, n in enumerate(DATA) : 
  if LIS[-1] < n: 
    LIS.append(n)
    l[i] = len(LIS) - 1
    MAX = l[i]
  else:
    l[i] = bisect.bisect_left(LIS, n)
    LIS[l[i]] = n

result = []
def r(i, n, ll) : #i부터 거꾸로 보면서 n을 찾는다.
  # print(i, n, ll)
  if i < 0 : return
  for j in range(i, -1, -1):
    if l[j] != n : continue #찾아야 하는 수
    if DATA[ll[-1]] <= DATA[j] : continue #증가하는 수열이면 안됨
    if n == 1 :
      x = ll+[j]
      lll = []
      for k in x[::-1] :
        lll.append(DATA[k]) #우선순위큐
      result.append(lll)
    r(j-1, n-1, ll+[j])

#재귀
for i in range(len(l)-1, 0, -1) :
  v = l[i]
  if v == MAX :
    r(i-1, v-1, [i])
  
result.sort()
if K > len(result) :
  print(-1)
else :
  print(*result[K-1])

- TLE, 재귀 함수가 시간복잡도 지수적으로 증가.
- [2, 1, 4, 3, 6, 5, ...] 의 입력은 2^(n/2)개 만큼의 LIS가 있다. 따라서 이러한 접근으로는 풀 수 없다.
  - 특히 n이 500인 경우 2^250개가 되므로, 이를 모두 풀기엔 메모리도 부족하게 된다.

### 풀이
- LIS의 개수를 구하는 함수를 만들고, LIS를 재구성 한다 O(N^2 log N)

In [ ]:
import collections
N, K = map(int, input().split())
L = [*map(int, input().split())]

M = [-1] * (N + 1) #메모
def LIS(s) : #L[s]에서 시작하는 IS중 최대 길이를 반환한다.
  ret = M[s + 1]
  if ret != -1 : return ret #항상 S[s]는 있기 때문에 길이는 최소 1이다.

  ret = 1
  for i in range(s + 1, N) :
    if s == -1 or L[s] < L[i] :
      ret = max(ret, LIS(i) + 1)
  return ret
    
def count(s): #S[s]에서 시작하는 LIS의 갯수를 반환한다.
  if LIS(s) == 1: return 1 #L[s]에서 시작하는 LIS가 1이면 자기 자신만 있다.
  ret = M[s + 1]
  if ret != -1 : return ret #메모이제이션

  ret = 0
  for i in range(s + 1, N) :
    if (s == -1 or L[s] < L[i]) and LIS(s) == LIS(i) + 1 :
      ret = min(2000000000, ret + count(i))
  return ret

def reconstruct(s, skip, lis: collections.deque) :
  if s != -1 : lis.appendleft(L[s])
  followers = collections.deque()
  for i in range(s + 1, N) :
    if (s == -1 or L[s] < L[i]) and LIS(s) == LIS(i) + 1 :
      followers.appendleft((L[i], i))
  followers = sorted(followers)

  for i in range(len(followers)) :
    idx = followers[i][1]
    cnt = count(idx)
    if cnt <= skip :
      skip -= cnt
    else :
      reconstruct(idx, skip, lis)
      break

x = collections.deque()
reconstruct(-1, K - 1, x)
if x :
  print(" ".join(map(str, reversed(x))))
else :
  print(-1)

- 아무래도 이것도 TLE가 나는 것 같다.